# get pandas df from fuseki

In [1]:
import pandas as pd
import json
import requests
import wget
import time
import os
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

In [3]:
q = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX : <http://h-da.de/fbi/artontology/>

SELECT ?artwork ?url ?name ?abstract (group_concat(?motif;separator=';') as ?motifs)
WHERE {
  ?artwork rdf:type :artwork;
     rdfs:label ?name;
     :image ?url;
     :abstract ?abstract;
     :motif/rdfs:label ?motif.
}
group by ?artwork ?url ?name ?abstract
"""

In [4]:
ds = "http://neuds.de:3030/artontology"
df = get_sparql_dataframe(ds, q)
df['id'] = df['artwork'].map(lambda x: x.split('/')[-1])
df.head()

,artwork,url,name,abstract,motifs,id
0,http://www.wikidata.org/entity/Q29019486,https://upload.wikimedia.org/wikipedia/commons...,Saint Proculus of Pozzuoli and his mother Sant...,Saints Proculus and Nicea is a 1636-1637 paint...,Nicea;Saint Procolo;Saint Procolo and Nicea,Q29019486
1,http://www.wikidata.org/entity/Q7527543,https://upload.wikimedia.org/wikipedia/commons...,Statue of John A. Macdonald,The Sir John A. Macdonald statue is a bronze s...,John A. Macdonald,Q7527543
2,http://www.wikidata.org/entity/Q3944494,https://upload.wikimedia.org/wikipedia/commons...,The Holy Family with Saint Catherine of Alexan...,Holy Family with Saint Catherine of Alexandria...,Virgin Mary;Catherine of Alexandria;boy;Child ...,Q3944494
3,http://www.wikidata.org/entity/Q368788,https://upload.wikimedia.org/wikipedia/commons...,Pietà,Pietà is a painting by the Italian Renaissance...,Virgin Mary;Jesus Christ;man;woman,Q368788
4,http://www.wikidata.org/entity/Q3630743,https://upload.wikimedia.org/wikipedia/commons...,Self-Portrait at the Age of 63,Self-Portrait at the Age of 63 is a self-portr...,Rembrandt;man,Q3630743


In [8]:
df[df['id'] == 'Q152851']['url'].values[0]

'https://upload.wikimedia.org/wikipedia/commons/d/d9/Eugene_de_Blaas_In_the_water.jpg'

# crawl artworks

In [39]:
def scale_url(url):
  art_url = url.replace('/commons/', '/commons/thumb/')
  art_name = art_url.split('/')[-1]
  art_url = art_url + '/512px-' + art_name
  return(art_url)

In [42]:
dir = 'res'
if not os.path.exists(dir):
    os.makedirs(dir)
data_fetch = df[['id', 'url']].values
for data in data_fetch:
  id_art = data[0]
  url_art = scale_url(data[1])
  file_type = url_art.split('.')[-1].lower()
  file_art = f'{dir}/{id_art}.{file_type}'
  print(f'Downloading: {id_art} -> {file_art}')
  os.system(f'wget -e robots=off -O {file_art} {url_art}')
  time.sleep(.1)
print('done')

Downloading: Q29019486 -> res/Q29019486.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Artemisia_gentileschi%2C_i_santi_procolo_e_nicea%2C_1631_ca..JPG/512px-Artemisia_gentileschi%2C_i_santi_procolo_e_nicea%2C_1631_ca..JPG
Downloading: Q7527543 -> res/Q7527543.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/2/2c/Macdonaldstat.jpg/512px-Macdonaldstat.jpg
Downloading: Q3944494 -> res/Q3944494.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Lotto%2C_sacra_famiglia_con_santa_caterina_d%27alessandria.jpg/512px-Lotto%2C_sacra_famiglia_con_santa_caterina_d%27alessandria.jpg
Downloading: Q368788 -> res/Q368788.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/0/0a/Pietro_Perugino_cat19.jpg/512px-Pietro_Perugino_cat19.jpg
Downloading: Q3630743 -> res/Q3630743.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/7/7c/Rembrandt_Harmensz._van_Rijn_135.jpg/512px-Rembrandt_Harmensz._van_Rijn_135.jpg
Downloading: Q5002068 -> res/Q5002068.jpg
https://up

Downloading: Q3933573 -> res/Q3933573.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/a/ad/Rubens%2C_resurrezione%2C_pitti.jpg/512px-Rubens%2C_resurrezione%2C_pitti.jpg
Downloading: Q7765607 -> res/Q7765607.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/Ivan_Grohar_-_Sejalec.jpg/512px-Ivan_Grohar_-_Sejalec.jpg
Downloading: Q3814745 -> res/Q3814745.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/0/0e/Peter_Paul_Rubens_014.jpg/512px-Peter_Paul_Rubens_014.jpg
Downloading: Q19947331 -> res/Q19947331.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Copie_d%27apr%C3%A8s_La_Barque_de_Dante_de_Delacroix_-_Edouard_Manet_%28B_830%29.jpg/512px-Copie_d%27apr%C3%A8s_La_Barque_de_Dante_de_Delacroix_-_Edouard_Manet_%28B_830%29.jpg
Downloading: Q22671120 -> res/Q22671120.jpeg
https://upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Behold_the_Bridegroom_Arriving.jpeg/512px-Behold_the_Bridegroom_Arriving.jpeg
Downloading: Q655420 -> res/Q655420.jpg
https:/

Downloading: Q11129389 -> res/Q11129389.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/4/4a/Perugino%2C_apparizione_della_vergine_a_san_bernardo%2C_monaco.jpg/512px-Perugino%2C_apparizione_della_vergine_a_san_bernardo%2C_monaco.jpg
Downloading: Q18511742 -> res/Q18511742.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/5/56/Pieter_de_Hooch_-_The_Visit_-_WGA11684.jpg/512px-Pieter_de_Hooch_-_The_Visit_-_WGA11684.jpg
Downloading: Q2488682 -> res/Q2488682.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/6/6a/Rotterdam_kunstwerk_Santa_Claus.jpg/512px-Rotterdam_kunstwerk_Santa_Claus.jpg
Downloading: Q3201475 -> res/Q3201475.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/c/c1/Pietro_Perugino_cat68.jpg/512px-Pietro_Perugino_cat68.jpg
Downloading: Q926536 -> res/Q926536.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/2/27/LionBelfort.jpg/512px-LionBelfort.jpg
Downloading: Q3947210 -> res/Q3947210.jpg
https://upload.wikimedia.org/wikipedia/commons/thu

Downloading: Q7492201 -> res/Q7492201.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/Tom_Roberts_-_Shearing_the_rams_-_Google_Art_Project.jpg/512px-Tom_Roberts_-_Shearing_the_rams_-_Google_Art_Project.jpg
Downloading: Q21648031 -> res/Q21648031.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/c/c6/Zelfportret_met_blauw_schetsboek%2C_L%C3%A9on_Spilliaert%2C_1907%2C_Koninklijk_Museum_voor_Schone_Kunsten_Antwerpen%2C_2697.jpg/512px-Zelfportret_met_blauw_schetsboek%2C_L%C3%A9on_Spilliaert%2C_1907%2C_Koninklijk_Museum_voor_Schone_Kunsten_Antwerpen%2C_2697.jpg
Downloading: Q7474637 -> res/Q7474637.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Cabildoabierto-Subercaseaux.jpg/512px-Cabildoabierto-Subercaseaux.jpg
Downloading: Q2350980 -> res/Q2350980.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Domenico_Veneziano_-_Pala_di_Santa_Lucia_dei_Magnoli_-_Google_Art_Project.jpg/512px-Domenico_Veneziano_-_Pala_di_Santa_Lucia_dei_Magnoli_-_Google_

Downloading: Q16169361 -> res/Q16169361.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/3/37/Theodore_Roosevelt_statue%2C_South_Park_Blocks%2C_January_2009.jpg/512px-Theodore_Roosevelt_statue%2C_South_Park_Blocks%2C_January_2009.jpg
Downloading: Q2627034 -> res/Q2627034.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/Caravaggio_-_Saint_Catherine_of_Alexandria_-_Google_Art_Project.jpg/512px-Caravaggio_-_Saint_Catherine_of_Alexandria_-_Google_Art_Project.jpg
Downloading: Q3698199 -> res/Q3698199.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/f/fc/Paolo_Uccello_015.jpg/512px-Paolo_Uccello_015.jpg
Downloading: Q3221334 -> res/Q3221334.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/5/57/Andrea_Mategna_-_Le_Christ_avec_l%27%C3%A2me_de_la_Vierge.jpg/512px-Andrea_Mategna_-_Le_Christ_avec_l%27%C3%A2me_de_la_Vierge.jpg
Downloading: Q3275305 -> res/Q3275305.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/b/be/Parmigianino_-_The_Vision_of_St_Jero

...

Downloading: Q1130734 -> res/Q1130734.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/8/89/Rembrandt_Harmensz._van_Rijn_011.jpg/512px-Rembrandt_Harmensz._van_Rijn_011.jpg
Downloading: Q13571594 -> res/Q13571594.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Rubens_-_El_rapto_de_Europa.jpg/512px-Rubens_-_El_rapto_de_Europa.jpg
Downloading: Q517408 -> res/Q517408.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/7/77/Amor-Psyche-Canova-JBU01.JPG/512px-Amor-Psyche-Canova-JBU01.JPG
Downloading: Q21175628 -> res/Q21175628.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/2/23/Night_Travellers_at_a_Cross_-_Ladislav_Medny%C3%A1nszky.jpg/512px-Night_Travellers_at_a_Cross_-_Ladislav_Medny%C3%A1nszky.jpg
Downloading: Q594406 -> res/Q594406.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Pieter_Bruegel_the_Elder_-_The_Fall_of_the_Rebel_Angels_-_Google_Art_Project.jpg/512px-Pieter_Bruegel_the_Elder_-_The_Fall_of_the_Rebel_Angels_-_Google_Art_Proj

Downloading: Q42305169 -> res/Q42305169.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Napol%C3%A9on_iii_en_uniforme_militaire%2C_peinture_d%27hippolyte_flandrin.jpg/512px-Napol%C3%A9on_iii_en_uniforme_militaire%2C_peinture_d%27hippolyte_flandrin.jpg
Downloading: Q57081221 -> res/Q57081221.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/a/a3/Jos%C3%A9_de_Ribera_053.jpg/512px-Jos%C3%A9_de_Ribera_053.jpg
Downloading: Q4343303 -> res/Q4343303.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/1/12/Peter_the_Great_Monument_SPB.jpg/512px-Peter_the_Great_Monument_SPB.jpg
Downloading: Q472799 -> res/Q472799.png
https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Mary%2C_Untier_of_Knots_by_Schmidtner.png/512px-Mary%2C_Untier_of_Knots_by_Schmidtner.png
Downloading: Q4448822 -> res/Q4448822.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/1/1b/Tizian_009.jpg/512px-Tizian_009.jpg
Downloading: Q775405 -> res/Q775405.jpg
https://upload.wikimedia.org/wikip

Downloading: Q315716 -> res/Q315716.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/Mathis_Gothart_Gr%C3%BCnewald_001.jpg/512px-Mathis_Gothart_Gr%C3%BCnewald_001.jpg
Downloading: Q47305137 -> res/Q47305137.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/3/35/Statue_of_Victoria_in_Victoria_Park%2C_Hong_Kong_1.jpg/512px-Statue_of_Victoria_in_Victoria_Park%2C_Hong_Kong_1.jpg
Downloading: Q7565810 -> res/Q7565810.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/8/82/Boer_adelaide.jpg/512px-Boer_adelaide.jpg
Downloading: Q5896517 -> res/Q5896517.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/Dem%C3%B3crito_y_Her%C3%A1clito%2C_de_Rubens_%28Museo_Nacional_de_Escultura_de_Valladolid%29.JPG/512px-Dem%C3%B3crito_y_Her%C3%A1clito%2C_de_Rubens_%28Museo_Nacional_de_Escultura_de_Valladolid%29.JPG
Downloading: Q22056331 -> res/Q22056331.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/Johans_Valters_-_Boys_Bathing_-_Google_Art_Project.jpg/512

Downloading: Q46239 -> res/Q46239.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/3/3e/Elide%2C_moneta_di_adriano_con_zeus_olimpio.JPG/512px-Elide%2C_moneta_di_adriano_con_zeus_olimpio.JPG
Downloading: Q2629559 -> res/Q2629559.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Fragonard%2C_The_Swing.jpg/512px-Fragonard%2C_The_Swing.jpg
Downloading: Q6065609 -> res/Q6065609.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Joaqu%C3%ADn_Sorolla_y_Bastida_-_Strolling_along_the_Seashore_-_Google_Art_Project.jpg/512px-Joaqu%C3%ADn_Sorolla_y_Bastida_-_Strolling_along_the_Seashore_-_Google_Art_Project.jpg
Downloading: Q6543739 -> res/Q6543739.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Jean-L%C3%A9on_G%C3%A9r%C3%B4me%2C_Phryne_revealed_before_the_Areopagus_%281861%29_-_01.jpg/512px-Jean-L%C3%A9on_G%C3%A9r%C3%B4me%2C_Phryne_revealed_before_the_Areopagus_%281861%29_-_01.jpg
Downloading: Q23898 -> res/Q23898.jpg
https://upload.wikimedia.org/wikip

Downloading: Q500533 -> res/Q500533.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/1/1c/Pojke_som_tittar_p%C3%A5_m%C3%A5nen_Bollhust%C3%A4ppan_flickr_brionv.jpg/512px-Pojke_som_tittar_p%C3%A5_m%C3%A5nen_Bollhust%C3%A4ppan_flickr_brionv.jpg
Downloading: Q65044996 -> res/Q65044996.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/2/21/2006-08-06_-_01_-_Road_Trip_-_Day_14_-_United_States_-_Oregon_-_Ashland_-_Lithia_Park_-_headless_abr_4889358640.jpg/512px-2006-08-06_-_01_-_Road_Trip_-_Day_14_-_United_States_-_Oregon_-_Ashland_-_Lithia_Park_-_headless_abr_4889358640.jpg
Downloading: Q5108945 -> res/Q5108945.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Antonello_da_Messina_-_Christ_at_the_Column_-_WGA0744.jpg/512px-Antonello_da_Messina_-_Christ_at_the_Column_-_WGA0744.jpg
Downloading: Q3879285 -> res/Q3879285.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/c/cc/Amedeo_Modigliani_063.jpg/512px-Amedeo_Modigliani_063.jpg
Downloading: Q3698260 -> res/Q

In [9]:
dft = df['id'].head()

In [43]:
def gen_dict(df):
  d = {'scores':[]}
  v = df.values
  for i, _ in enumerate(v):
    for j in range(i+1, len(v)):
      d['scores'].append([v[i], v[j], {}])
  return(d)

def gen_bocv(df):
  d = {}
  for aid in df:
    d[aid] = {}
  return(d)

In [45]:
d = gen_bocv(df['id'])
with open('art_bocv.json', 'w') as f:
    json.dump(d, f)

In [10]:
d = gen_dict(df['id'])
with open('art_sim.json', 'w') as f:
    json.dump(d, f)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
for x in d['scores']:
  t1 = 'Q3697654'
  t2 = 'Q3952498'
  if t1 in x or t2 in x:
    print(x)

['Q29019486', 'Q3697654', {}]
['Q29019486', 'Q3952498', {}]
['Q7527543', 'Q3697654', {}]
['Q7527543', 'Q3952498', {}]
['Q3944494', 'Q3697654', {}]
['Q3944494', 'Q3952498', {}]
['Q368788', 'Q3697654', {}]
['Q368788', 'Q3952498', {}]
['Q3630743', 'Q3697654', {}]
['Q3630743', 'Q3952498', {}]
['Q5002068', 'Q3697654', {}]
['Q5002068', 'Q3952498', {}]
['Q17579092', 'Q3697654', {}]
['Q17579092', 'Q3952498', {}]
['Q50324305', 'Q3697654', {}]
['Q50324305', 'Q3952498', {}]
['Q17480508', 'Q3697654', {}]
['Q17480508', 'Q3952498', {}]
['Q2234397', 'Q3697654', {}]
['Q2234397', 'Q3952498', {}]
['Q57450255', 'Q3697654', {}]
['Q57450255', 'Q3952498', {}]
['Q18684713', 'Q3697654', {}]
['Q18684713', 'Q3952498', {}]
['Q934458', 'Q3697654', {}]
['Q934458', 'Q3952498', {}]
['Q177647', 'Q3697654', {}]
['Q177647', 'Q3952498', {}]
['Q18736732', 'Q3697654', {}]
['Q18736732', 'Q3952498', {}]
['Q3212142', 'Q3697654', {}]
['Q3212142', 'Q3952498', {}]
['Q3842587', 'Q3697654', {}]
['Q3842587', 'Q3952498', {}]
['Q630

['Q3395790', 'Q3697654', {}]
['Q3395790', 'Q3952498', {}]
['Q15103953', 'Q3697654', {}]
['Q15103953', 'Q3952498', {}]
['Q3360743', 'Q3697654', {}]
['Q3360743', 'Q3952498', {}]
['Q548141', 'Q3697654', {}]
['Q548141', 'Q3952498', {}]
['Q970808', 'Q3697654', {}]
['Q970808', 'Q3952498', {}]
['Q24971410', 'Q3697654', {}]
['Q24971410', 'Q3952498', {}]
['Q12046321', 'Q3697654', {}]
['Q12046321', 'Q3952498', {}]
['Q5719359', 'Q3697654', {}]
['Q5719359', 'Q3952498', {}]
['Q2420918', 'Q3697654', {}]
['Q2420918', 'Q3952498', {}]
['Q24269542', 'Q3697654', {}]
['Q24269542', 'Q3952498', {}]
['Q3472935', 'Q3697654', {}]
['Q3472935', 'Q3952498', {}]
['Q5689366', 'Q3697654', {}]
['Q5689366', 'Q3952498', {}]
['Q17015401', 'Q3697654', {}]
['Q17015401', 'Q3952498', {}]
['Q17509039', 'Q3697654', {}]
['Q17509039', 'Q3952498', {}]
['Q2535482', 'Q3697654', {}]
['Q2535482', 'Q3952498', {}]
['Q30594133', 'Q3697654', {}]
['Q30594133', 'Q3952498', {}]
['Q6106762', 'Q3697654', {}]
['Q6106762', 'Q3952498', {}]
['Q7

['Q735061', 'Q3697654', {}]
['Q735061', 'Q3952498', {}]
['Q19966602', 'Q3697654', {}]
['Q19966602', 'Q3952498', {}]
['Q727875', 'Q3697654', {}]
['Q727875', 'Q3952498', {}]
['Q3234637', 'Q3697654', {}]
['Q3234637', 'Q3952498', {}]
['Q29383342', 'Q3697654', {}]
['Q29383342', 'Q3952498', {}]
['Q3973084', 'Q3697654', {}]
['Q3973084', 'Q3952498', {}]
['Q2325834', 'Q3697654', {}]
['Q2325834', 'Q3952498', {}]
['Q3978277', 'Q3697654', {}]
['Q3978277', 'Q3952498', {}]
['Q1133821', 'Q3697654', {}]
['Q1133821', 'Q3952498', {}]
['Q21451665', 'Q3697654', {}]
['Q21451665', 'Q3952498', {}]
['Q650086', 'Q3697654', {}]
['Q650086', 'Q3952498', {}]
['Q3207987', 'Q3697654', {}]
['Q3207987', 'Q3952498', {}]
['Q2410932', 'Q3697654', {}]
['Q2410932', 'Q3952498', {}]
['Q7916475', 'Q3697654', {}]
['Q7916475', 'Q3952498', {}]
['Q4478998', 'Q3697654', {}]
['Q4478998', 'Q3952498', {}]
['Q3815141', 'Q3697654', {}]
['Q3815141', 'Q3952498', {}]
['Q20087546', 'Q3697654', {}]
['Q20087546', 'Q3952498', {}]
['Q19904884'

['Q23929396', 'Q3697654', {}]
['Q23929396', 'Q3952498', {}]
['Q16599636', 'Q3697654', {}]
['Q16599636', 'Q3952498', {}]
['Q20170619', 'Q3697654', {}]
['Q20170619', 'Q3952498', {}]
['Q2712679', 'Q3697654', {}]
['Q2712679', 'Q3952498', {}]
['Q3697599', 'Q3697654', {}]
['Q3697599', 'Q3952498', {}]
['Q3769183', 'Q3697654', {}]
['Q3769183', 'Q3952498', {}]
['Q18162030', 'Q3697654', {}]
['Q18162030', 'Q3952498', {}]
['Q7744868', 'Q3697654', {}]
['Q7744868', 'Q3952498', {}]
['Q2736379', 'Q3697654', {}]
['Q2736379', 'Q3952498', {}]
['Q5296650', 'Q3697654', {}]
['Q5296650', 'Q3952498', {}]
['Q19892199', 'Q3697654', {}]
['Q19892199', 'Q3952498', {}]
['Q18384815', 'Q3697654', {}]
['Q18384815', 'Q3952498', {}]
['Q536024', 'Q3697654', {}]
['Q536024', 'Q3952498', {}]
['Q4400244', 'Q3697654', {}]
['Q4400244', 'Q3952498', {}]
['Q23815653', 'Q3697654', {}]
['Q23815653', 'Q3952498', {}]
['Q22976991', 'Q3697654', {}]
['Q22976991', 'Q3952498', {}]
['Q518940', 'Q3697654', {}]
['Q518940', 'Q3952498', {}]
['

['Q6728427', 'Q3952498', {}]
['Q5712840', 'Q3697654', {}]
['Q5712840', 'Q3952498', {}]
['Q22976966', 'Q3697654', {}]
['Q22976966', 'Q3952498', {}]
['Q1169874', 'Q3697654', {}]
['Q1169874', 'Q3952498', {}]
['Q3204090', 'Q3697654', {}]
['Q3204090', 'Q3952498', {}]
['Q2293936', 'Q3697654', {}]
['Q2293936', 'Q3952498', {}]
['Q7437304', 'Q3697654', {}]
['Q7437304', 'Q3952498', {}]
['Q7729586', 'Q3697654', {}]
['Q7729586', 'Q3952498', {}]
['Q52257225', 'Q3697654', {}]
['Q52257225', 'Q3952498', {}]
['Q1132723', 'Q3697654', {}]
['Q1132723', 'Q3952498', {}]
['Q22329904', 'Q3697654', {}]
['Q22329904', 'Q3952498', {}]
['Q20171155', 'Q3697654', {}]
['Q20171155', 'Q3952498', {}]
['Q19361652', 'Q3697654', {}]
['Q19361652', 'Q3952498', {}]
['Q2537695', 'Q3697654', {}]
['Q2537695', 'Q3952498', {}]
['Q874058', 'Q3697654', {}]
['Q874058', 'Q3952498', {}]
['Q22055510', 'Q3697654', {}]
['Q22055510', 'Q3952498', {}]
['Q567833', 'Q3697654', {}]
['Q567833', 'Q3952498', {}]
['Q17342608', 'Q3697654', {}]
['Q17

...

['Q21747156', 'Q3952498', {}]
['Q24939394', 'Q3697654', {}]
['Q24939394', 'Q3952498', {}]
['Q24657567', 'Q3697654', {}]
['Q24657567', 'Q3952498', {}]
['Q17030912', 'Q3697654', {}]
['Q17030912', 'Q3952498', {}]
['Q1213917', 'Q3697654', {}]
['Q1213917', 'Q3952498', {}]
['Q7677949', 'Q3697654', {}]
['Q7677949', 'Q3952498', {}]
['Q636001', 'Q3697654', {}]
['Q636001', 'Q3952498', {}]
['Q23012270', 'Q3697654', {}]
['Q23012270', 'Q3952498', {}]
['Q3842397', 'Q3697654', {}]
['Q3842397', 'Q3952498', {}]
['Q7435638', 'Q3697654', {}]
['Q7435638', 'Q3952498', {}]
['Q922648', 'Q3697654', {}]
['Q922648', 'Q3952498', {}]
['Q7715890', 'Q3697654', {}]
['Q7715890', 'Q3952498', {}]
['Q19924628', 'Q3697654', {}]
['Q19924628', 'Q3952498', {}]
['Q18749621', 'Q3697654', {}]
['Q18749621', 'Q3952498', {}]
['Q1087249', 'Q3697654', {}]
['Q1087249', 'Q3952498', {}]
['Q2713823', 'Q3697654', {}]
['Q2713823', 'Q3952498', {}]
['Q3842589', 'Q3697654', {}]
['Q3842589', 'Q3952498', {}]
['Q7934017', 'Q3697654', {}]
['Q79

['Q3664360', 'Q3952498', {}]
['Q38036678', 'Q3697654', {}]
['Q38036678', 'Q3952498', {}]
['Q2015484', 'Q3697654', {}]
['Q2015484', 'Q3952498', {}]
['Q22443298', 'Q3697654', {}]
['Q22443298', 'Q3952498', {}]
['Q978987', 'Q3697654', {}]
['Q978987', 'Q3952498', {}]
['Q6034865', 'Q3697654', {}]
['Q6034865', 'Q3952498', {}]
['Q6046671', 'Q3697654', {}]
['Q6046671', 'Q3952498', {}]
['Q18573172', 'Q3697654', {}]
['Q18573172', 'Q3952498', {}]
['Q690427', 'Q3697654', {}]
['Q690427', 'Q3952498', {}]
['Q10908882', 'Q3697654', {}]
['Q10908882', 'Q3952498', {}]
['Q326887', 'Q3697654', {}]
['Q326887', 'Q3952498', {}]
['Q3922376', 'Q3697654', {}]
['Q3922376', 'Q3952498', {}]
['Q1104588', 'Q3697654', {}]
['Q1104588', 'Q3952498', {}]
['Q20199477', 'Q3697654', {}]
['Q20199477', 'Q3952498', {}]
['Q5610806', 'Q3697654', {}]
['Q5610806', 'Q3952498', {}]
['Q17279024', 'Q3697654', {}]
['Q17279024', 'Q3952498', {}]
['Q19912411', 'Q3697654', {}]
['Q19912411', 'Q3952498', {}]
['Q5150039', 'Q3697654', {}]
['Q515

['Q3850319', 'Q3952498', {}]
['Q1219263', 'Q3697654', {}]
['Q1219263', 'Q3952498', {}]
['Q3819135', 'Q3697654', {}]
['Q3819135', 'Q3952498', {}]
['Q1122002', 'Q3697654', {}]
['Q1122002', 'Q3952498', {}]
['Q24037779', 'Q3697654', {}]
['Q24037779', 'Q3952498', {}]
['Q16038514', 'Q3697654', {}]
['Q16038514', 'Q3952498', {}]
['Q1422666', 'Q3697654', {}]
['Q1422666', 'Q3952498', {}]
['Q94006', 'Q3697654', {}]
['Q94006', 'Q3952498', {}]
['Q3889223', 'Q3697654', {}]
['Q3889223', 'Q3952498', {}]
['Q17320369', 'Q3697654', {}]
['Q17320369', 'Q3952498', {}]
['Q15644879', 'Q3697654', {}]
['Q15644879', 'Q3952498', {}]
['Q28952743', 'Q3697654', {}]
['Q28952743', 'Q3952498', {}]
['Q1133420', 'Q3697654', {}]
['Q1133420', 'Q3952498', {}]
['Q2714237', 'Q3697654', {}]
['Q2714237', 'Q3952498', {}]
['Q25415024', 'Q3697654', {}]
['Q25415024', 'Q3952498', {}]
['Q19912543', 'Q3697654', {}]
['Q19912543', 'Q3952498', {}]
['Q3204212', 'Q3697654', {}]
['Q3204212', 'Q3952498', {}]
['Q5560923', 'Q3697654', {}]
['Q5

['Q3697621', 'Q3697654', {}]
['Q3697621', 'Q3952498', {}]
['Q3646498', 'Q3697654', {}]
['Q3646498', 'Q3952498', {}]
['Q4657634', 'Q3697654', {}]
['Q4657634', 'Q3952498', {}]
['Q619265', 'Q3697654', {}]
['Q619265', 'Q3952498', {}]
['Q4009550', 'Q3697654', {}]
['Q4009550', 'Q3952498', {}]
['Q2609585', 'Q3697654', {}]
['Q2609585', 'Q3952498', {}]
['Q26237885', 'Q3697654', {}]
['Q26237885', 'Q3952498', {}]
['Q20169896', 'Q3697654', {}]
['Q20169896', 'Q3952498', {}]
['Q19820361', 'Q3697654', {}]
['Q19820361', 'Q3952498', {}]
['Q2983399', 'Q3697654', {}]
['Q2983399', 'Q3952498', {}]
['Q22443796', 'Q3697654', {}]
['Q22443796', 'Q3952498', {}]
['Q1652981', 'Q3697654', {}]
['Q1652981', 'Q3952498', {}]
['Q570188', 'Q3697654', {}]
['Q570188', 'Q3952498', {}]
['Q5532298', 'Q3697654', {}]
['Q5532298', 'Q3952498', {}]
['Q29021696', 'Q3697654', {}]
['Q29021696', 'Q3952498', {}]
['Q19924882', 'Q3697654', {}]
['Q19924882', 'Q3952498', {}]
['Q3842771', 'Q3697654', {}]
['Q3842771', 'Q3952498', {}]
['Q159

['Q66538187', 'Q3697654', {}]
['Q66538187', 'Q3952498', {}]
['Q27954929', 'Q3697654', {}]
['Q27954929', 'Q3952498', {}]
['Q3842476', 'Q3697654', {}]
['Q3842476', 'Q3952498', {}]
['Q51052495', 'Q3697654', {}]
['Q51052495', 'Q3952498', {}]
['Q4146206', 'Q3697654', {}]
['Q4146206', 'Q3952498', {}]
['Q18552705', 'Q3697654', {}]
['Q18552705', 'Q3952498', {}]
['Q22257349', 'Q3697654', {}]
['Q22257349', 'Q3952498', {}]
['Q3224275', 'Q3697654', {}]
['Q3224275', 'Q3952498', {}]
['Q3842568', 'Q3697654', {}]
['Q3842568', 'Q3952498', {}]
['Q23646694', 'Q3697654', {}]
['Q23646694', 'Q3952498', {}]
['Q6738074', 'Q3697654', {}]
['Q6738074', 'Q3952498', {}]
['Q2535486', 'Q3697654', {}]
['Q2535486', 'Q3952498', {}]
['Q5221207', 'Q3697654', {}]
['Q5221207', 'Q3952498', {}]
['Q4717953', 'Q3697654', {}]
['Q4717953', 'Q3952498', {}]
['Q7628395', 'Q3697654', {}]
['Q7628395', 'Q3952498', {}]
['Q29027068', 'Q3697654', {}]
['Q29027068', 'Q3952498', {}]
['Q4556666', 'Q3697654', {}]
['Q4556666', 'Q3952498', {}]
